In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn' заглушить SettingWithCopyWarning

### Выгрузка данных

In [28]:
#датасет из слака
data_slack=pd.read_csv('raw_data.csv',low_memory=False,header=[0, 1])

In [3]:
#датасет с соревнования 
dataset = pd.read_csv('train.zip', index_col=0, header=[0, 1])
dataset.rename(
    columns={
        'Unnamed: 209_level_1': 'count',
        'Unnamed: 210_level_1': 'price',
    },
    level = 1,
    inplace = True
)
data_competition=dataset.copy()

In [29]:
del dataset

In [4]:
data_competition['price_in_60s']=data_competition['price']['price'].shift(60).fillna(0)
data_competition=data_competition[data_competition.price_in_60s!=0]

In [5]:
data_competition['y']=np.log(data_competition['price_in_60s']/data_competition['price']['price'])

In [67]:
data_slack.head()

timestamp_second bid_price_0                                        \
  Unnamed: 0_level_1     nunique       mean   median       std     min   
0                  0           7  44.183379  44.1865  0.007340  44.171   
1                  1           1  44.190000  44.1900  0.000000  44.190   
2                  2           3  44.187467  44.1860  0.001767  44.186   
3                  3           3  44.181038  44.1860  0.006993  44.173   
4                  4           2  44.213095  44.2190  0.012474  44.188   

                          bid_quantity_0  ... sell_trade_quantity              \
      max   first    last        nunique  ...                 sum mean median   
0  44.190  44.171  44.190              5  ...                 0.0  NaN    NaN   
1  44.190  44.190  44.190              1  ...                 0.0  NaN    NaN   
2  44.190  44.190  44.186              3  ...                 0.0  NaN    NaN   
3  44.188  44.186  44.188              2  ...                 0.0  NaN    NaN   
4  44.219  44.188  44.219              2  ...                 0.0  NaN    NaN   

                                        count                    y  
  std min max first last Unnamed: 207_level_1 Unnamed: 208_level_1  
0 NaN NaN NaN   NaN  NaN                   66            -0.065894  
1 NaN NaN NaN   NaN  NaN                   13            -0.066870  
2 NaN NaN NaN   NaN  NaN                   15            -0.064833  
3 NaN NaN NaN   NaN  NaN                   52            -0.040764  
4 NaN NaN NaN   NaN  NaN                   21            -0.063813  

[5 rows x 209 columns]

In [30]:
#data=data_competition
data=data_slack.copy()

## Сборка признаков

#### Таблица с используемыми признаками:
(Benchmark Dataset for Mid-Price Forecasting of Limit Order Book Data with Machine Learning Methods)

пока реализованы $u_{1},u_{2},u_{3},u_{4}$

![title](features.png)

Basic

In [50]:
#5 levels LOB
U1=pd.DataFrame()
for i in range(5):
    U1[f'ask_price_level_{i+1}']=data[f'ask_price_{i}']['mean']
    U1[f'ask_quantity_{i+1}']=data[f'ask_quantity_{i}']['sum']
    
    U1[f'bid_price_level_{i+1}']=data[f'bid_price_{i}']['mean']
    U1[f'bid_quantity_{i+1}']=data[f'bid_quantity_{i}']['sum']#

Time-Insensitive

In [51]:
#Spread & mid-price
U2=pd.DataFrame()
for i in range(1,6,1):
    U2[f'spread_{i}'] = U1[f'ask_price_level_{i}']-U1[f'bid_price_level_{i}']
    U2[f'mid_price_{i}'] = (U1[f'ask_price_level_{i}'] + U1[f'bid_price_level_{i}'])/2

In [52]:
#price differences
U3=pd.DataFrame()
for i in range(2,6,1):
    U3['ask_price_diff'] = U1['ask_price_level_5'] - U1['ask_price_level_1']
    U3['bid_price_diff'] = U1['bid_price_level_1'] - U1[f'bid_price_level_5']
    U3[f'abs_ask_price_diff_{i}'] = abs(U1[f'ask_price_level_{i}']-\
                                    U1[f'ask_price_level_{i-1}'])
    U3[f'abs_bid_price_diff_{i}'] = abs(U1[f'bid_price_level_{i}']-\
                                    U1[f'bid_price_level_{i-1}'])

In [53]:
#Price & volume mean
U4=pd.DataFrame()
for i in range(1,6,1):
    U4['mean_price_ask'] = (
        U1['ask_price_level_1']+U1['ask_price_level_2']+\
        U1['ask_price_level_3']+U1['ask_price_level_4']+\
        U1['ask_price_level_5'] )/5
    U4['mean_price_bid'] = (
        U1['bid_price_level_1']+U1['bid_price_level_2']+\
        U1['bid_price_level_3']+U1['bid_price_level_4']+\
        U1['bid_price_level_5'] )/5
    
    U4['mean_vol_ask'] = (
        U1['ask_quantity_1']+U1['ask_quantity_2']+\
        U1['ask_quantity_3']+U1['ask_quantity_4']+\
        U1['ask_quantity_5'] )/5
    U4['mean_vol_bid'] = (
        U1['bid_quantity_1']+U1['bid_quantity_2']+\
        U1['bid_quantity_3']+U1['bid_quantity_4']+\
        U1['bid_quantity_5'] )/5

In [54]:
#Accumulated Differences
U5_1=pd.DataFrame()
for i in range(1,6,1):
    U5_1[f'price_diff_level_{i}'] = U1[f'ask_price_level_{i}']-\
        U1[f'bid_price_level_{i}']
    U5_1[f'vol_diff_level_{i}'] = U1[f'ask_quantity_{i}']-\
        U1[f'bid_quantity_{i}']

In [55]:
U5=pd.DataFrame()
U5['cum_price_diff'] = U5_1['price_diff_level_1'] + U5_1['price_diff_level_2']+\
    U5_1['price_diff_level_3'] + U5_1['price_diff_level_4']+\
    U5_1['price_diff_level_5']

U5['cum_vol_diff'] = U5_1['vol_diff_level_1'] + U5_1['vol_diff_level_2']+\
    U5_1['vol_diff_level_3'] + U5_1['vol_diff_level_4']+\
    U5_1['vol_diff_level_5']

## Построение моделей

#### Линейная регрессия с L2 регуляризацией

In [37]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

In [56]:
X=U1.join(U2).join(U3).join(U4).join(U5)
y=data.y['Unnamed: 208_level_1'].rename('y').to_frame().y
#y=data['y']

In [57]:
del U1,U2,U3,U4,U5

In [58]:
#expand X by 1.0 column
X.insert(0, "intercept", 1)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [60]:
ridge=Ridge(fit_intercept=False)

In [61]:
ridge.fit(X_train,y_train)

Ridge(fit_intercept=False)

In [62]:
y_pred=pd.Series(ridge.predict(X_test))

In [63]:
#fig,ax=plt.subplots(figsize=(12,8))
##ax1=y_test.plot(label='fact')
#ax2=y_pred.plot(label='pred')
#plt.legend()

In [64]:
from sklearn.metrics import r2_score,mean_squared_error

Результат на датасете из слака:

In [65]:
r2_score(y_test,y_pred)

0.008138032182456567

In [66]:
mean_squared_error(y_test,y_pred)

0.004971121935029167

Результат на датасете с соревнования:

In [25]:
r2_score(y_test,y_pred)

0.9999991173925106

In [26]:
mean_squared_error(y_test,y_pred)

2.1686369920301095e-05

#### Регрессия случайным лесом

In [132]:
from sklearn.ensemble import RandomForestRegressor

In [134]:
from sklearn.model_selection import GridSearchCV

In [ ]:
dtr=RandomForestRegressor(random_state=0)

In [135]:
parameters = {
    #'n_estimators': [50,75],
    'max_depth': [1,2,3,4],
}

In [136]:
#grid_search=GridSearchCV(dtr, parameters)

In [137]:
#долго бежит
#grid_search.fit(X_train,y_train)

GridSearchCV(estimator=RandomForestRegressor(random_state=0),
             param_grid={'max_depth': [1, 2, 3, 4]})

In [138]:
grid_search.best_params_

{'max_depth': 4}

In [139]:
best_clf=grid_search.best_estimator_

In [140]:
dtr.fit(X_train,y_train)

RandomForestRegressor(random_state=0)

In [141]:
y_pred_best=best_clf.predict(X_test)

In [142]:
y_pred=pd.Series(dtr.predict(X_test))

Результат на случайном лесе со стандартными параметрами (датасет из слэка):

In [143]:
r2_score(y_test,y_pred)

0.31946552864731115

In [144]:
mean_squared_error(y_test,y_pred)

0.0034413360553631604

Результат на случайном лесе с максимальной глубиной 4:

In [148]:
r2_score(y_test,y_pred_best)

0.029113875078265572

In [147]:
mean_squared_error(y_test,y_pred_best)

0.004909590282331825